In [5]:
"""
Extract gridMET variables 
Google Earth Engine (GEE) Python API
Author: maxwell.cook@colorado.edu
"""

import ee
import geemap
import time

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')
print("GEE Authenticated !")

GEE Authenticated !


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [30]:
def monitor_export(task, timeout):
    """ Monitors EE export task """
    while task.active():
        print('Waiting for export to finish..\n\tPatience young padawan.')
        time.sleep(timeout)  
    
    # Get the status of the task
    status = task.status()
    
    # Check if the task failed or succeeded
    if status['state'] == 'COMPLETED':
        print("Export completed successfully !!!!")
    elif status['state'] == 'FAILED':
        print(f"Export failed! Bummer. Reason: {status.get('error_message', 'Unknown error')}")
    else:
        print(f"Export ended with state: {status['state']}")

print("Functions loaded !")

Functions loaded !


In [7]:
# Load fire perimeters
fires = ee.FeatureCollection('projects/jfsp-aspen/assets/nifc_aspenfires_2018to2023')
print(f"Number of aspen fires: {fires.size().getInfo()}")
print(fires.first().propertyNames().getInfo())

Number of aspen fires: 50
['Fire_Year', 'Fire_ID', 'Ig_Date', 'Last_Date', 'Fire_Name', 'system:index']


In [ ]:
# Load the gridmet image collection

In [8]:
gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')
print(f"gridMET bands available for analysis:\n\n{gridmet.first().bandNames().getInfo()}")

gridMET bands available for analysis:

['pr', 'rmax', 'rmin', 'sph', 'srad', 'th', 'tmmn', 'tmmx', 'vs', 'erc', 'eto', 'bi', 'fm100', 'fm1000', 'etr', 'vpd']


In [9]:
# calculate the 15 year average
mean15 = gridmet.filter(ee.Filter.calendarRange(2009,2024,'year')).mean() 
gridmet.first().bandNames().getInfo()

['pr',
 'rmax',
 'rmin',
 'sph',
 'srad',
 'th',
 'tmmn',
 'tmmx',
 'vs',
 'erc',
 'eto',
 'bi',
 'fm100',
 'fm1000',
 'etr',
 'vpd']

In [ ]:
# Load the AFD 

In [10]:
afds = ee.FeatureCollection('projects/jfsp-aspen/assets/AFD/vnp14img_geo_srm_pix_area_aspenfires')
print(f"Number of AFD: {afds.size().getInfo()}")
print(afds.first().propertyNames().getInfo())

Number of AFD: 51853
['Fire_ID', 'acq_date', 'daynight', 'Ig_Date', 'Last_Date', 'afdID', 'system:index']


In [11]:
# get a list of unique active fire days
fire_days = afds.aggregate_array('acq_date').getInfo()
fire_days = set(fire_days)
print(f"Number of 'fire days': {len(fire_days)}")

Number of 'fire days': 390


In [25]:
# map over the list of unique dates
def gridmet_summary(fire):
    """ calculate the daily gridmet for active fire detections """
    
    # grab the fire id
    fire_id = fire.get('Fire_ID')
    # filter obs. to the day of burning
    afds_ = afds.filter(ee.Filter.eq('Fire_ID',fire_id))
    # get the unique dates
    fire_days = ee.List(afds_.aggregate_array('acq_date')).distinct()
    
    def get_daily(day):
        """ calculate the daily gridmet summary"""
        # filter to this day's detections
        afds_day = afds_.filter(ee.Filter.eq('acq_date',day))
        bounds = afds_day.geometry().bounds() # gather the bounds for this days detections
        # retrieve the date information for filtering
        acq_date = ee.Date(day) # the day of acquisition
        prev_day = acq_date.advance(-1,'days') # previous day for calculating the mean
        
        # calculate the gridmet mean
        gridmet_ = gridmet.filterDate(prev_day, acq_date).select(attrs)
        gridmet_ = gridmet_.mean() # the average prev day to day of burn

        # calculation the deviation from the 15-year average
        anomaly = gridmet_.subtract(mean15.select(attrs)).rename([atr+"_dev" for atr in attrs])

        # run the reduction
        stats = gridmet_.addBands(anomaly).reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=bounds,
            scale=4000,
            bestEffort=True
        )

        return ee.Feature(None, stats.set("acq_date", day))

    results = fire_days.map(get_daily)
    return ee.FeatureCollection(results)

print("Function ready !")

Function ready !


In [26]:
# list of gridmet attributes
attrs = ['fm1000', 'vpd', 'erc', 'vs']
        
fire_stats = fires.limit.map(gridmet_summary).flatten()

print("Number of features:", fire_stats.first().getInfo())
print("Submitted !")

Number of features: 798
Submitted !


In [29]:
fire_stats.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '00000000000000000000_0',
 'properties': {'acq_date': '2018-06-09',
  'erc': 92.08238884045336,
  'erc_dev': 43.29555197518705,
  'fm1000': 5.952092446341345,
  'fm1000_dev': -6.854169788210935,
  'vpd': 1.8013252251649174,
  'vpd_dev': 0.9771088598703653,
  'vs': 4.224280634234028,
  'vs_dev': 0.16459902183013886}}

In [31]:
# Export the table. 
fire_stats = fire_stats.map(lambda ftr: ftr.setGeometry(None)) # drop geometry column

export_task = ee.batch.Export.table.toDrive(
    collection=fire_stats,
    description='vn14img_gridmet',
    fileFormat='CSV', 
    fileNamePrefix='vn14img_gridmet',
    folder='GRIDMET'
)

# Start the export task
export_task.start()
print("Export to Earth Engine Asset started!")
# Monitor the task until it's finished
monitor_export(export_task, 60) 

Export to Earth Engine Asset started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting fo